In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [2]:
# CIFAR-10 dataset Loading
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
# OnehotEncoding, converting labels to categorical
num_outputclasses = 10
y_train = to_categorical(y_train, num_outputclasses)
y_test = to_categorical(y_test, num_outputclasses)

In [5]:
# Image preprocessing
def preprocess_images(images):
    images = images.astype("float32") / 255.0
    return images

In [6]:
# Preprocessing - CNN
x_trainCNN = preprocess_images(x_train)
x_testCNN = preprocess_images(x_test)

In [7]:
# Image Generator-CNN
image_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

In [9]:
# CNN model defining
def CNNModel_build():
    model = models.Sequential([
        layers.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_outputclasses, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

custom_model_CNN = CNNModel_build()
custom_model_CNN.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 160,202 (625.79 KB)

 Trainable params: 160,202 (625.79 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# CNN Model Training
CNNTrain_log = custom_model_CNN.fit(image_generator.flow(x_trainCNN, y_train, batch_size=64),
                                    epochs=10, validation_data=(x_testCNN, y_test))

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 45ms/step - accuracy: 0.2975 - loss: 1.8871 - val_accuracy: 0.5049 - val_loss: 1.3753
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.4852 - loss: 1.4322 - val_accuracy: 0.5675 - val_loss: 1.2049
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 40ms/step - accuracy: 0.5455 - loss: 1.2693 - val_accuracy: 0.6071 - val_loss: 1.1164
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.5862 - loss: 1.1721 - val_accuracy: 0.6234 - val_loss: 1.0684
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 40ms/step - accuracy: 0.6149 - loss: 1.1007 - val_accuracy: 0.6692 - val_loss: 0.9510
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.6352 - loss: 1.0379 - val_accuracy: 0.6650 - val_loss: 0.9602
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.6489 - loss: 1.0043 - val_accuracy: 0.6888 - val_loss: 0.9020
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.6658 - loss: 0.9554 - val_accurac

In [11]:
# Image preprocessing - ResNet(Images 224x224)
def resnetdata_preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.image.random_flip_left_right(image)
    image = tf.image.rot90(image)
    image = preprocess_input(image)
    return image, label

In [12]:
# CIFAR-10 data Converting to TensorFlow dataset (with preprocessing)
datasetTrain = tf.data.Dataset.from_tensor_slices((x_train, y_train))
datasetTrain = datasetTrain.map(resnetdata_preprocess).batch(64).prefetch(tf.data.experimental.AUTOTUNE)

datasetTest = tf.data.Dataset.from_tensor_slices((x_test, y_test))
datasetTest = datasetTest.map(resnetdata_preprocess).batch(64).prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
# ResNet50 Model(PreTrained) loading
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [17]:
# custom layers adding for classification
resnet_model_custom = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_outputclasses, activation='softmax')
])

resnet_model_custom.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model_custom.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,851,274 (90.99 MB)

 Trainable params: 263,562 (1.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [18]:
# ResNet50 model Training
resnetTrain_log = resnet_model_custom.fit(datasetTrain, epochs=10, validation_data=datasetTest)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 242s 288ms/step - accuracy: 0.7077 - loss: 0.8415 - val_accuracy: 0.7949 - val_loss: 0.5785
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 247s 280ms/step - accuracy: 0.8168 - loss: 0.5209 - val_accuracy: 0.8119 - val_loss: 0.5221
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 232s 296ms/step - accuracy: 0.8376 - loss: 0.4528 - val_accuracy: 0.8284 - val_loss: 0.4824
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 249s 280ms/step - accuracy: 0.8550 - loss: 0.4145 - val_accuracy: 0.8306 - val_loss: 0.4802
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 220s 281ms/step - accuracy: 0.8612 - loss: 0.3865 - val_accuracy: 0.8321 - val_loss: 0.4727
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 255s 273ms/step - accuracy: 0.8759 - loss: 0.3549 - val_accuracy: 0.8297 - val_loss: 0.4851
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 218s 279ms/step - accuracy: 0.8799 - loss: 0.3341 - val_accuracy: 0.8318 - val_loss: 0.4986
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 261s 277ms/step - accuracy: 0.8884 -

In [19]:
# Evaluating Model
def evaluate_model(model, name, dataset):
    loss, acc = model.evaluate(dataset, verbose=0)
    print(f'{name} Accuracy: {acc:.4f}')

evaluate_model(custom_model_CNN, 'CNN', tf.data.Dataset.from_tensor_slices((x_testCNN, y_test)).batch(64))
evaluate_model(resnet_model_custom, 'ResNet50', datasetTest)

CNN Accuracy: 0.7008
ResNet50 Accuracy: 0.8306


In [20]:
# Predictions
predict_cnn = custom_model_CNN.predict(x_testCNN).argmax(axis=1)
predict_resnet = resnet_model_custom.predict(datasetTest).argmax(axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 44s 251ms/step


In [21]:
# test labels coverting to class indices
y_testLabels = y_test.argmax(axis=1)

In [22]:
# Classification Report
print("Performance - CNN Model :")
print(classification_report(y_testLabels, predict_cnn))

print("Performance - ResNet50 Model:")
print(classification_report(y_testLabels, predict_resnet))

Performance - CNN Model :
              precision    recall  f1-score   support

           0       0.72      0.76      0.74      1000
           1       0.75      0.87      0.80      1000
           2       0.80      0.41      0.54      1000
           3       0.63      0.43      0.51      1000
           4       0.71      0.62      0.66      1000
           5       0.68      0.56      0.62      1000
           6       0.62      0.89      0.73      1000
           7       0.74      0.76      0.75      1000
           8       0.77      0.85      0.81      1000
           9       0.65      0.85      0.74      1000

    accuracy                           0.70     10000
   macro avg       0.71      0.70      0.69     10000
weighted avg       0.71      0.70      0.69     10000

Performance - ResNet50 Model:
              precision    recall  f1-score   support

           0       0.81      0.91      0.86      1000
           1       0.94      0.86      0.90      1000
           2       0.8